In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Read in merged CSV file
merge = pd.read_csv('merged_files.csv')
print(merge.columns)
print(merge['batting_team'].unique())
merge.head()

In [ ]:
# We will use only data from the last 4 years (2020-2024)
merge[['start-year', 'start-day', 'start-month']] = merge['start_date'].str.split('-', expand=True)
df = merge.loc[merge['start-year'].astype(int) >= 2020]
merge = merge.drop(['start-year', 'start-day', 'start-month'], axis=1)
print(df.shape)
df.head()

In [ ]:
# Create dataframe of matches only from teams participating in the 2024 World Cup
with open('teams.txt') as file:
    teams_list = file.read().splitlines()
wc20=df.loc[(df['bowling_team'].isin(teams_list)) | (df['batting_team'].isin(teams_list))]
print(wc20.shape)
wc20.head()

In [ ]:
group_A = teams_list[0:5]
group_B = teams_list[5:10]
group_C = teams_list[10:15]
group_D = teams_list[15:20]

### Calculate Stats

In [ ]:
_df = (df
       .loc[:, ['striker', 'runs_off_bat', 'extras']]
       .groupby(['striker'], as_index = False)
       .sum())
# _df_sorted = _df.sort_values(by='runs_off_bat', ascending=False)

In [ ]:
num_bowls = (df
       .loc[:, ['striker', 'bowling_team', 'batting_team']]
       .groupby(['striker', 'batting_team'], as_index = False)
       .count()
       .rename(columns = {'bowling_team' : 'n_bowls'}))
df2 = _df.merge(num_bowls, on = ['striker'])

In [ ]:
# Calculate runs per bowl for strikers
df2['runs_per_bowl'] = (df2['runs_off_bat'] + df2['extras']) / df2['n_bowls']
df2['weighted_runs_per_bowl'] = df2['runs_per_bowl'] * df2['n_bowls']
df2.sort_values(by = 'weighted_runs_per_bowl', ascending = False)
print(df2.columns)

In [ ]:
us = df2[df2['batting_team'] == 'United States of America']
us.sort_values(by = 'weighted_runs_per_bowl', ascending = False)

In [ ]:
# TODO: Calculate runs_conceded values

In [ ]:
players = pd.read_csv('players.csv')

# Add runs per bowl values to players.csv (for hitters & all-rounders)
# Ignore rpb values for bowlers for now
players = players.merge(df2[['striker', 'runs_per_bowl']], how='left', left_on='name',right_on='striker')
players = players.drop(columns = ['striker'])

# TODO: Add runs_conceded values to players.csv 

In [ ]:
players.head(7)
#pd.set_option('display.max_rows', None)
#players

### Run Simulations

In [ ]:
# TODO: implement simulations based on the calculated player data